In [222]:
import pandas as pd
import myslack
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
import pickle
import time
import re
from tqdm import tqdm
from tqdm import trange

import requests
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [223]:
driver = webdriver.Chrome()  
url = 'https://www.yogiyo.co.kr/mobile/?utm_source=google&utm_medium=cpc&utm_campaign=sem_bra_orignal&utm_term=%EC%9A%94%EA%B8%B0%EC%9A%94&utm_id=sem_000001&referrer=adjust_tracker%3Dgrm2aa%26adjust_google_network%3Dg%26adjust_google_placement%3D%26adjust_campaign%3Dsem_bra_web_kr_700886066_cpc%26adjust_adgroup%3D38249737818%26adjust_creative%3D%EC%9A%94%EA%B8%B0%EC%9A%94_b&gclid=EAIaIQobChMIv9DaxIL-3AIVA6yWCh0C8glmEAAYASAAEgLbF_D_BwE#/%EC%84%9C%EC%9A%B8/138223/'
driver.get(url) 

In [224]:
food_dict = {
    '프랜차이즈':3, '치킨':4, '피자':5, '양식':5, '중국집':6,
    '한식':7, '일식':8, '돈가스':8, '족발':9, '보쌈':9, 
    '야식':10, '분식':11, '카페':12, '디저트':12 }

In [225]:
# 입력한 위치로 설정하기
def set_location(location):
    print(location+'으로 위치 설정 하는중...')
    driver.find_element_by_css_selector('#search > div > form > input').click()
    driver.find_element_by_css_selector('#button_search_address > button.btn-search-location-cancel.btn-search-location.btn.btn-default > span').click()
    driver.find_element_by_css_selector('#search > div > form > input').send_keys(location)
    driver.find_element_by_css_selector('#button_search_address > button.btn.btn-default.ico-pick').click()
    time.sleep(1)
    print(location+'으로 위치 설정 완료!')
    
# 카테고리 페이지로 넘어가기
def go_to_category(category):
    print(category+' 카테고리 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(food_dict.get(category))).click()
    time.sleep(2)
    print(category+' 카테고리 페이지 로드 완료!')

# 해당 카테고리의 모든 음식점 객체들을 리턴
def get_all_restaurant_elements():
    restaurants = driver.find_elements_by_css_selector('#content > div > div.restaurant-list > div.col-sm-6.contract')
    return restaurants

# 해당 카테고리의 모든 음식점 갯수를 리턴
def get_all_restaurant_count():
    restaurant_counts = int(driver.find_element_by_css_selector('#restaurant_count').text)
    return restaurant_counts

# 해당 음식점의 이름을 리턴
def get_restaurant_name(restaurant):
    restaurant_name = restaurant.find_element_by_class_name('restaurant-name').text
    return restaurant_name

# 해당 카테고리의 음식점 페이지로 넘어가기    
def go_to_restaurant(restaurant):
    restaurant_name = get_restaurant_name(restaurant)
    print(restaurant_name+' 음식점 페이지 로드중...')
    restaurant.click()
    time.sleep(3) 
    print(restaurant_name+' 음식점 페이지 로드 완료!')
    
# 해당 음식점의 리뷰 페이지로 넘어가기
def go_to_review():
    print('리뷰 페이지로 로드중...')
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    time.sleep(1)
    print('리뷰 페이지로 로드 완료!')
    
# 해당 음식점의 모든 리뷰 객체들
def get_all_review_elements():
    reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
    return reviews
    
# 페이지 한번 맨아래로 내리기
def scroll_bottom():
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

# 카테고리(음식점 리스트) 페이지에서 음식점 리스트 로드하기
def cat_page_scroll_repeat(i):
    # 브라우저의 스크롤 위치 조절
    height=200+118*(i//2)
    print('카테고리(음식점 리스트) 페이지 (0,{}) 위치로 내리는중...'.format(height))
    driver.execute_script("window.scrollTo(0,{})".format(height))
    time.sleep(3)
    print('카테고리(음식점 리스트) 페이지 (0,{}) 위치로 내리기 완료!'.format(height))
    
# 더보기 클릭하기 
def click_more_review():
    driver.find_element_by_class_name('btn-more').click()
    time.sleep(2)

# 리뷰 페이지 모두 펼치기
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in trange(click_count):
        try:
            scroll_bottom()
            click_more_review()
        except Exception as e:
            pass
    scroll_bottom()
    print('모든 리뷰 불러오기 완료!')
        
# 페이지 뒤로 가기 (한 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감)
def go_back_page():
    print('페이지 돌아가기중...')
    driver.execute_script("window.history.go(-1)")
    time.sleep(3)
    print('페이지 돌아가기 완료!')


In [219]:
cat_page_scroll_repeat(0)

카테고리(음식점 리스트) 페이지 (0,200) 위치로 내리는중...
카테고리(음식점 리스트) 페이지 (0,200) 위치로 내리기 완료!


In [220]:
len(get_all_restaurant_elements())

20

In [221]:
for i, restaurant in enumerate(get_all_restaurant_elements()): 
    try:
        print(i,restaurant)
        cat_page_scroll_repeat(i) 
        go_to_restaurant(restaurant) 
        go_back_page() 
    except Exception as e:
        print(e)
        pass

0 <selenium.webdriver.remote.webelement.WebElement (session="68e806577b24087e05bc85d8da9dc7a4", element="0.7309326761676302-1")>
카테고리(음식점 리스트) 페이지 (0,200) 위치로 내리는중...
카테고리(음식점 리스트) 페이지 (0,200) 위치로 내리기 완료!
7번가피자-잠실새내점 음식점 페이지 로드중...
7번가피자-잠실새내점 음식점 페이지 로드 완료!
페이지 돌아가기중...
페이지 돌아가기 완료!
1 <selenium.webdriver.remote.webelement.WebElement (session="68e806577b24087e05bc85d8da9dc7a4", element="0.7309326761676302-2")>
카테고리(음식점 리스트) 페이지 (0,200) 위치로 내리는중...
카테고리(음식점 리스트) 페이지 (0,200) 위치로 내리기 완료!
Message: stale element reference: element is not attached to the page document
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)

2 <selenium.webdriver.remote.webelement.WebElement (session="68e806577b24087e05bc85d8da9dc7a4", element="0.7309326761676302-3")>
카테고리(음식점 리스트) 페이지 (0,318) 위치로 내리는중...
카테고리(음식점 리스트) 페이지 (0,318) 위치로 내리기 완료!
Message: stale element reference: element is not attached to the pag

KeyboardInterrupt: 

In [183]:
def yogiyo_crawling(location, category):
    set_location(location) 
    go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감

    df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
                               'Total','Taste','Quantity','Delivery','Date'])

    print('Start {} - {} Crawling...'.format(location, category))
    for restaurant, i in tqdm(enumerate(get_all_restaurant_elements())): # 해당 카테고리의 음식점 개수만큼 loop
        try:
            cat_page_scroll_repeat(i) # i번째 음식점 위치로 page를 내림
            go_to_restaurant(restaurant, i) # i번째 음식점 페이지로 넘어감
            go_to_review() # 해당 음식점의 리뷰페이지로 넘어감
            stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴

            for review in tqdm(get_all_review_elements()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
                try:
                    df.loc[len(df)] = { 
                        'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
                        'UserID':review.find_element_by_css_selector('span.review-id.ng-binding').text,
                        'Menu':review.find_element_by_css_selector('div.order-items.default.ng-binding').text,
                        'Review':review.find_element_by_css_selector('p').text,
                        'Total':str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))),
                        'Taste':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text,
                        'Quantity':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text,
                        'Delivery':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(9)').text,
                        'Date':review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text,
                    }
                except Exception as e:
                    print('리뷰 페이지 에러')
                    print(e)
                    pass
                
            print('음식점 리스트 페이지로 돌아가는중...')
            go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
            time.sleep(2)
            
        except Exception as e:
            print('음식점 리스트 페이지 에러')
            print(e)
            pass
        
    print('End of {} {} Crawling!'.format(location, category))
    myslack.send_slack('{} {} crawling finish!!!'.format(location, category))
    return df

In [184]:
%time chinese_df = yogiyo_crawling('방이2동','치킨')

방이2동으로 위치 설정 하는중...
방이2동으로 위치 설정 완료!
치킨 카테고리 페이지 로드중...


20it [00:00, 24513.76it/s]

치킨 카테고리 페이지 로드 완료!
Start 방이2동 - 치킨 Crawling...
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' and 'int'
음식점 리스트 페이지 에러
unsupported operand type(s) for //: 'WebElement' a

<Response [200]>
CPU times: user 46.6 ms, sys: 10.4 ms, total: 57 ms
Wall time: 4.43 s


In [ ]:
chicken_df.tail()

In [226]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기

---